In [1]:

import math
def InitialDataReducedMass(orbitalangle,r0):
    phi=orbitalangle
    radiusseparation=r0 #separation one hundred
    eccentricity=np.zeros(2)
    mass=np.ones(2)
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    masses=mass #*masssun (natural units)
    return phi, radiusseparation,eccentricity, masses
    

In [2]:
def getxyuveqreducedmass(initdat): #using reduced mass cooordinate system
    phi,radiusseparation,eccentricity, masses=initdatreducedmass
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    x0=np.zeros(2)
    y0=np.zeros(2)
    x0[1]=radiusseparation*np.cos(phi)*metersperAU
    y0[1]=radiusseparation*np.sin(phi)*metersperAU
    if phi==0:
        print("zero")
        x0[1]=radiusseparation
        y0[1]=0
    if phi==math.pi:
        print("pi")
        x0[1]=-radiusseparation
        y0[1]=0
    z0=np.zeros(2)
    print("x0", x0)
    print("y0", y0)
    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2) #*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    masstotal=np.sum(masses)
    reducedmass[0]=masstotal
    print(masses)
    for i in np.arange(1):
        j=(i+1)%2 #reverse masses
        reducedmass[i+1]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    metersperAU=1 #natural units
    rphys=np.zeros(len(masses))
    for i in np.arange(2):
        #rphys is r in the reduced mass system
        rphys[i]=radiusseparation #*metersperAU*reducedmass[i]/masses[i]
    x0[0]=0.0
    x0phys=rphys*x0/radiusseparation
    x0phys[0]=0.0
    y0phys=rphys*y0/radiusseparation
    z0phys=np.zeros(2)
    r0=np.sqrt(x0**2+y0**2)
    print(rphys)
    #G=1
    Gconstant=1
    F=(Gconstant*masstotal**2/(2*rphys)**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*masstotal/(rphys))
    print(v)
    print("r0", r0)
    ux0=-v*y0/radiusseparation #[1]
    uy0=v*x0/radiusseparation #[1]
        
    #ux0=-v*y0/r0
    #uy0=v*x0/r0 #initial data in y only
    #evolve in plane only
    #circular orbit
    uz0=np.zeros(2)
    
    a=Gconstant*masstotal/((2*rphys)**2)
    ax0=-a*x0/radiusseparation
    ay0=-a*y0/radiusseparation
    az0=np.zeros(2)
    
    statevec=[]
    avec=[]
    for i in np.arange(len(x0phys)):
        stateveci=np.array([x0phys[i],y0phys[i],z0phys[i],ux0[i],uy0[i],uz0[i]])
        aveci=np.array([ax0[i],ay0[i],az0[i]])
        statevec.append(stateveci)
        avec.append(aveci)
    statevecnp=np.array(statevec)
    avecnp=np.array(avec)
    return masstotal, reducedmass,statevecnp,avecnp

In [3]:
import random,numpy as np
initdatreducedmass=InitialDataReducedMass(0,50)

In [4]:
xyuvaeqreducedmass=getxyuveqreducedmass(initdatreducedmass)
print(xyuvaeqreducedmass)
masstotal, reducedmass,xvec,avec=xyuvaeqreducedmass

zero
x0 [ 0. 50.]
y0 [0. 0.]
[1. 1.]
[2.  0.5]
[50. 50.]
[0.0004 0.0004]
[0.2 0.2]
r0 [ 0. 50.]
(2.0, array([2. , 0.5]), array([[ 0. ,  0. ,  0. , -0. ,  0. ,  0. ],
       [50. ,  0. ,  0. , -0. ,  0.2,  0. ]]), array([[-0.    , -0.    ,  0.    ],
       [-0.0002, -0.    ,  0.    ]]))


In [5]:
xvec[:,2]

array([0., 0.])

In [6]:
axii=np.zeros(len(xvec[:,0]))
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
rii

array([ 0., 50.])

In [7]:
def RK4implicit(h,t,xvec,f): #not a finite difference so no step in y
    k1= h*f(t,xvec)
    k2=h*f(t+h/2, xvec+k1/2)
    k3=h*f(t+h/2,xvec+k2/2)
    k4=h*f(t+h,xvec+k3)
    return t+h, xvec+1/6.*(k1+2.*k2+2.*k3+k4)

In [8]:
class OrbitDiffEq:
    def __init__(self,reducedmass,xvec,avec,t0):
        self.reducedmass=reducedmass
        self.xvec=xvec
        self.ti=t0
        self.avec=avec
    def dxidt(self,t,xvec):
        return xvec[:,3].transpose()
    def dyidt(self,t,xvec):
        return xvec[:,4].transpose()
    def dzidt(self,t,xvec):
        return xvec[:,5].transpose()
    def dvxidt(self,t,xvec):
        #return axi[m]
        axii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    axii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    axii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,0]  - xvec[k,0])/rreljk**3
        return axii
    def dvyidt(self,t,xvec):
        #return axi[m]
        ayii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    ayii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    ayii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,1]  - xvec[k,1])/rreljk**3
        return ayii
    def dvzidt(self,t,xvec):
        #return axi[m]
        azii=np.zeros(len(xvec[:,0]))
        rii=np.sqrt(xvec[:,0]**2+xvec[:,1]**2+xvec[:,2]**2)
        Gconstant=1 #6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(1,len(rii)):
                if j==0:
                    azii[j]=0.0
                if j!=k:
                    rreljk=np.abs((xvec[j,0] - xvec[k,0])**2+(xvec[j,1]-xvec[k,1])**2+(xvec[j,2]-xvec[k,2])**2)**(1./2.)
                    azii[j]-=Gconstant*self.reducedmass[k]*(xvec[j,2]  - xvec[k,2])/rreljk**3
        return azii
    
    def dvecdt(self,t,xvec):
        avec=np.array([self.dvxidt(self,xvec),self.dvyidt(self,xvec),self.dvzidt(self,xvec)])
        avecT=avec.transpose()
        self.avec=avecT
        dvec2=np.array([self.dxidt(t,xvec),self.dyidt(t,xvec),self.dzidt(self,xvec),avec[0,:],avec[1,:],avec[2,:]])
                       
        dvec2T=dvec2.transpose()
        return dvec2T
    def updateINTERNAL(self,xvecii,tii):
        self.xvec=xvecii
        self.ti=tii
        return self
    def update(self,xvecii,avecii,tii):
        self.xvec=xvecii
        self.avec=avecii,
        self.ti=tii
    def print2D(self):
        print(self.reducedmass,self.xvec[1], self.ti)
        return self
    def list2D(self):
        return self.reducedmass,self.xvec,self.avec,self.ti
    def timestepRK4ODE(self,step,dt):

    
        h=dt
        #tnew,ynew, intval=RK4(h,t,y,f)
        #m represents choices of mass
        i=step
        
        tnew,intvalxvec=RK4implicit(h,self.ti,self.xvec,self.dvecdt)
        #xii = intvalx
        #tnew,intvalvx=RK4implicit(h,self.ti,self.vxi,self.dvxidt)
        #vxii=intvalvx
        #tnew,intvaly=RK4implicit(h,self.ti,self.yi,self.dyidt)
        #yii = intvaly
        #tnew,intvalvy=RK4implicit(h,self.ti,self.vyi,self.dvyidt)
        #vyii=intvalvy
        #tnew,intvalz=RK4implicit(h,self.ti,self.zi,self.dzidt)
        #zii = intvalz
        #tnew,intvalvz=RK4implicit(h,self.ti,self.vzi,self.dvzidt)
        #vzii=intvalvz
        #axii=self.dvxidt(self.ti,np.zeros(np.shape(self.xi)))
        #ayii=self.dvyidt(self.ti,np.zeros(np.shape(self.yi)))
        #azii=self.dvzidt(self.ti,np.zeros(np.shape(self.zi)))
 
        #print(xii)
        self.updateINTERNAL(intvalxvec,tnew)
        return reducedmass, self.xvec,self.avec,self.ti

In [9]:
mtotal,reducedmass,xvec,avec=xyuvaeqreducedmass
ODEeq= OrbitDiffEq(reducedmass,xvec,avec,0.0)
ODEeq.print2D()
ODEeq.print2D()

[2.  0.5] [50.   0.   0.  -0.   0.2  0. ] 0.0
[2.  0.5] [50.   0.   0.  -0.   0.2  0. ] 0.0


In [10]:
dt=1 #natural units #*31556926 #seconds per year
numsteps=3000
mtotal, reducedmass,xvec0,avec0=xyuvaeqreducedmass
ODEeq= OrbitDiffEq(reducedmass,xvec0,avec0,0.0)
ODEeq.print2D()
t=0.0
star1x=[]
star2x=[]
star1a=[]
star2a=[]
times=[]
for i in np.arange(1,numsteps):
    reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
    ODEeq.print2D()
    star1x.append(xvec[0])
    star2x.append(xvec[1])
    star1a.append(avec[0])
    star2a.append(avec[1])
    times.append(t)



[2.  0.5] [50.   0.   0.  -0.   0.2  0. ] 0.0
[2.  0.5] [ 4.99996000e+01  1.99999467e-01  0.00000000e+00 -7.99997867e-04
  1.99998400e-01  0.00000000e+00] 1.0
[2.  0.5] [ 4.99984000e+01  3.99995733e-01  0.00000000e+00 -1.59998293e-03
  1.99993600e-01  0.00000000e+00] 2.0
[2.  0.5] [ 4.9996400e+01  5.9998560e-01  0.0000000e+00 -2.3999424e-03
  1.9998560e-01  0.0000000e+00] 3.0
[2.  0.5] [ 4.99936001e+01  7.99965867e-01  0.00000000e+00 -3.19986347e-03
  1.99974401e-01  0.00000000e+00] 4.0
[2.  0.5] [ 4.99900003e+01  9.99933335e-01  0.00000000e+00 -3.99973334e-03
  1.99960001e-01  0.00000000e+00] 5.0
[2.  0.5] [ 4.99856007e+01  1.19988480e+00  0.00000000e+00 -4.79953921e-03
  1.99942403e-01  0.00000000e+00] 6.0
[2.  0.5] [ 4.99804013e+01  1.39981707e+00  0.00000000e+00 -5.59926830e-03
  1.99921605e-01  0.00000000e+00] 7.0
[2.  0.5] [ 4.99744022e+01  1.59972695e+00  0.00000000e+00 -6.39890779e-03
  1.99897609e-01  0.00000000e+00] 8.0
[2.  0.5] [ 4.99676035e+01  1.79961123e+00  0.00000000e+

[2.  0.5] [43.78289221 24.1466012   0.         -0.0965864   0.17513157  0.        ] 126.0
[2.  0.5] [43.6859558  24.32153913  0.         -0.09728616  0.17474382  0.        ] 127.0
[2.  0.5] [43.58832042 24.49608792  0.         -0.09798435  0.17435328  0.        ] 128.0
[2.  0.5] [43.48998762 24.67024477  0.         -0.09868098  0.17395995  0.        ] 129.0
[2.  0.5] [43.39095898 24.84400689  0.         -0.09937603  0.17356384  0.        ] 130.0
[2.  0.5] [43.29123609 25.01737151  0.         -0.10006949  0.17316494  0.        ] 131.0
[2.  0.5] [43.19082055 25.19033586  0.         -0.10076134  0.17276328  0.        ] 132.0
[2.  0.5] [43.08971394 25.36289716  0.         -0.10145159  0.17235886  0.        ] 133.0
[2.  0.5] [42.98791791 25.53505265  0.         -0.10214021  0.17195167  0.        ] 134.0
[2.  0.5] [42.88543407 25.70679958  0.         -0.1028272   0.17154174  0.        ] 135.0
[2.  0.5] [42.78226406 25.87813521  0.         -0.10351254  0.17112906  0.        ] 136.0
[2.  0.5] 

[2.  0.5] [31.70315499 38.66406554  0.         -0.15465626  0.12681262  0.        ] 221.0
[2.  0.5] [31.54824551 38.79056851  0.         -0.15516227  0.12619298  0.        ] 222.0
[2.  0.5] [31.39283126 38.91645083  0.         -0.1556658   0.12557133  0.        ] 223.0
[2.  0.5] [31.23691473 39.04171049  0.         -0.15616684  0.12494766  0.        ] 224.0
[2.  0.5] [31.08049841 39.16634548  0.         -0.15666538  0.12432199  0.        ] 225.0
[2.  0.5] [30.92358481 39.29035381  0.         -0.15716142  0.12369434  0.        ] 226.0
[2.  0.5] [30.76617642 39.4137335   0.         -0.15765493  0.12306471  0.        ] 227.0
[2.  0.5] [30.60827578 39.53648257  0.         -0.15814593  0.1224331   0.        ] 228.0
[2.  0.5] [30.4498854  39.65859905  0.         -0.1586344   0.12179954  0.        ] 229.0
[2.  0.5] [30.29100783 39.780081    0.         -0.15912032  0.12116403  0.        ] 230.0
[2.  0.5] [30.1316456  39.90092647  0.         -0.15960371  0.12052658  0.        ] 231.0
[2.  0.5] 

[2.  0.5] [13.18212331 48.23102347  0.         -0.19292409  0.05272849  0.        ] 326.0
[2.  0.5] [12.98909427 48.28336598  0.         -0.19313346  0.05195638  0.        ] 327.0
[2.  0.5] [12.79585741 48.33493595  0.         -0.19333974  0.05118343  0.        ] 328.0
[2.  0.5] [12.60241582 48.38573256  0.         -0.19354293  0.05040966  0.        ] 329.0
[2.  0.5] [12.40877258 48.43575501  0.         -0.19374302  0.04963509  0.        ] 330.0
[2.  0.5] [12.21493081 48.48500248  0.         -0.19394001  0.04885972  0.        ] 331.0
[2.  0.5] [ 1.20208936e+01  4.85334742e+01  0.00000000e+00 -1.94133897e-01
  4.80835744e-02  0.00000000e+00] 332.0
[2.  0.5] [ 1.18266641e+01  4.85811694e+01  0.00000000e+00 -1.94324677e-01
  4.73066562e-02  0.00000000e+00] 333.0
[2.  0.5] [ 1.16322453e+01  4.86280873e+01  0.00000000e+00 -1.94512349e-01
  4.65289811e-02  0.00000000e+00] 334.0
[2.  0.5] [ 1.14376404e+01  4.86742271e+01  0.00000000e+00 -1.94696908e-01
  4.57505616e-02  0.00000000e+00] 335.0


[2.  0.5] [-3.05827350e+00  4.99063820e+01  0.00000000e+00 -1.99625528e-01
 -1.22330940e-02  0.00000000e+00] 408.0
[2.  0.5] [-3.25787403e+00  4.98937497e+01  0.00000000e+00 -1.99574999e-01
 -1.30314961e-02  0.00000000e+00] 409.0
[2.  0.5] [-3.45742243e+00  4.98803191e+01  0.00000000e+00 -1.99521276e-01
 -1.38296897e-02  0.00000000e+00] 410.0
[2.  0.5] [-3.65691552e+00  4.98660904e+01  0.00000000e+00 -1.99464361e-01
 -1.46276621e-02  0.00000000e+00] 411.0
[2.  0.5] [-3.85635009e+00  4.98510638e+01  0.00000000e+00 -1.99404255e-01
 -1.54254004e-02  0.00000000e+00] 412.0
[2.  0.5] [-4.05572296e+00  4.98352397e+01  0.00000000e+00 -1.99340959e-01
 -1.62228919e-02  0.00000000e+00] 413.0
[2.  0.5] [-4.25503095e+00  4.98186181e+01  0.00000000e+00 -1.99274472e-01
 -1.70201238e-02  0.00000000e+00] 414.0
[2.  0.5] [-4.45427085e+00  4.98011995e+01  0.00000000e+00 -1.99204798e-01
 -1.78170834e-02  0.00000000e+00] 415.0
[2.  0.5] [-4.65343948e+00  4.97829841e+01  0.00000000e+00 -1.99131936e-01
 -1.8

...

[2.  0.5] [-43.66702138  24.35551773   0.          -0.09742207  -0.17466809
   0.        ] 2229.0
[2.  0.5] [-43.76409386  24.18065526   0.          -0.09672262  -0.17505638
   0.        ] 2230.0
[2.  0.5] [-43.86046611  24.00540591   0.          -0.09602162  -0.17544186
   0.        ] 2231.0
[2.  0.5] [-43.95613659  23.82977247   0.          -0.09531909  -0.17582455
   0.        ] 2232.0
[2.  0.5] [-44.05110378  23.65375776   0.          -0.09461503  -0.17620442
   0.        ] 2233.0
[2.  0.5] [-44.14536615  23.47736458   0.          -0.09390946  -0.17658146
   0.        ] 2234.0
[2.  0.5] [-44.23892219  23.30059577   0.          -0.09320238  -0.17695569
   0.        ] 2235.0
[2.  0.5] [-44.33177042  23.12345415   0.          -0.09249382  -0.17732708
   0.        ] 2236.0
[2.  0.5] [-44.42390933  22.94594255   0.          -0.09178377  -0.17769564
   0.        ] 2237.0
[2.  0.5] [-44.51533747  22.76806382   0.          -0.09107226  -0.17806135
   0.        ] 2238.0
[2.  0.5] [-44.60605

[2.  0.5] [-4.97257905e+01  5.22931749e+00  0.00000000e+00 -2.09172700e-02
 -1.98903162e-01  0.00000000e+00] 2330.0
[2.  0.5] [-4.97463099e+01  5.03037303e+00  0.00000000e+00 -2.01214921e-02
 -1.98985240e-01  0.00000000e+00] 2331.0
[2.  0.5] [-4.97660334e+01  4.83134807e+00  0.00000000e+00 -1.93253923e-02
 -1.99064133e-01  0.00000000e+00] 2332.0
[2.  0.5] [-4.97849606e+01  4.63224582e+00  0.00000000e+00 -1.85289833e-02
 -1.99139842e-01  0.00000000e+00] 2333.0
[2.  0.5] [-4.98030912e+01  4.43306945e+00  0.00000000e+00 -1.77322778e-02
 -1.99212365e-01  0.00000000e+00] 2334.0
[2.  0.5] [-4.98204250e+01  4.23382215e+00  0.00000000e+00 -1.69352886e-02
 -1.99281700e-01  0.00000000e+00] 2335.0
[2.  0.5] [-4.98369617e+01  4.03450711e+00  0.00000000e+00 -1.61380285e-02
 -1.99347847e-01  0.00000000e+00] 2336.0
[2.  0.5] [-4.98527010e+01  3.83512752e+00  0.00000000e+00 -1.53405101e-02
 -1.99410804e-01  0.00000000e+00] 2337.0
[2.  0.5] [-4.98676426e+01  3.63568657e+00  0.00000000e+00 -1.45427463e-

   0.        ] 2464.0
[2.  0.5] [-45.33882251 -21.08058759   0.           0.08432235  -0.18135529
   0.        ] 2465.0
[2.  0.5] [-45.25413768 -21.26177375   0.           0.0850471   -0.18101655
   0.        ] 2466.0
[2.  0.5] [-45.16872878 -21.44261972   0.           0.08577048  -0.18067492
   0.        ] 2467.0
[2.  0.5] [-45.08259718 -21.62312262   0.           0.08649249  -0.18033039
   0.        ] 2468.0
[2.  0.5] [-44.99574426 -21.80327954   0.           0.08721312  -0.17998298
   0.        ] 2469.0
[2.  0.5] [-44.90817141 -21.98308761   0.           0.08793235  -0.17963269
   0.        ] 2470.0
[2.  0.5] [-44.81988003 -22.16254395   0.           0.08865018  -0.17927952
   0.        ] 2471.0
[2.  0.5] [-44.73087153 -22.34164569   0.           0.08936658  -0.17892349
   0.        ] 2472.0
[2.  0.5] [-44.64114734 -22.52038997   0.           0.09008156  -0.17856459
   0.        ] 2473.0
[2.  0.5] [-44.55070889 -22.69877392   0.           0.0907951   -0.17820284
   0.        ] 2474.

   0.        ] 2655.0
[2.  0.5] [-18.15413673 -46.58784519   0.           0.18635138  -0.07261655
   0.        ] 2656.0
[2.  0.5] [-17.96764061 -46.66008884   0.           0.18664036  -0.07187056
   0.        ] 2657.0
[2.  0.5] [-17.78085701 -46.73158593   0.           0.18692634  -0.07112343
   0.        ] 2658.0
[2.  0.5] [-17.59378892 -46.80233532   0.           0.18720934  -0.07037516
   0.        ] 2659.0
[2.  0.5] [-17.40643933 -46.87233587   0.           0.18748934  -0.06962576
   0.        ] 2660.0
[2.  0.5] [-17.21881123 -46.94158646   0.           0.18776635  -0.06887524
   0.        ] 2661.0
[2.  0.5] [-17.03090764 -47.01008599   0.           0.18804034  -0.06812363
   0.        ] 2662.0
[2.  0.5] [-16.84273155 -47.07783336   0.           0.18831133  -0.06737093
   0.        ] 2663.0
[2.  0.5] [-16.65428597 -47.14482749   0.           0.18857931  -0.06661714
   0.        ] 2664.0
[2.  0.5] [-16.46557393 -47.21106729   0.           0.18884427  -0.0658623
   0.        ] 2665.0

[2.  0.5] [ 13.26069272 -48.20948069   0.           0.19283792   0.05304277
   0.        ] 2816.0
[2.  0.5] [ 13.45342404 -48.15605239   0.           0.19262421   0.0538137
   0.        ] 2817.0
[2.  0.5] [ 13.64594011 -48.10185359   0.           0.19240741   0.05458376
   0.        ] 2818.0
[2.  0.5] [ 13.83823784 -48.04688516   0.           0.19218754   0.05535295
   0.        ] 2819.0
[2.  0.5] [ 14.03031416 -47.99114798   0.           0.19196459   0.05612126
   0.        ] 2820.0
[2.  0.5] [ 14.222166   -47.93464294   0.           0.19173857   0.05688866
   0.        ] 2821.0
[2.  0.5] [ 14.41379029 -47.87737096   0.           0.19150948   0.05765516
   0.        ] 2822.0
[2.  0.5] [ 14.60518395 -47.81933293   0.           0.19127733   0.05842074
   0.        ] 2823.0
[2.  0.5] [ 14.79634393 -47.7605298    0.           0.19104212   0.05918538
   0.        ] 2824.0
[2.  0.5] [ 14.98726717 -47.70096249   0.           0.19080385   0.05994907
   0.        ] 2825.0
[2.  0.5] [ 15.177950

In [11]:
npstar2x=np.array(star2x)
npstar1x=np.array(star1x)
npstar2a=np.array(star2a)
npstar1a=np.array(star1a)
nptimes=np.array(times)

In [12]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="Circular orbit, RK4", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(npstar1x[:,0],npstar1x[:,1], size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.line(npstar2x[:,0],npstar2x[:,1], line_color="blue", legend="mu")


p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...

In [13]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="Circular orbit, RK4", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(times,npstar1a[:,0], size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(times,npstar2a[:,0], line_color="blue", legend="mu")

p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...

In [14]:
def RunFirstOrbit(dt,initcondition,printoutput):
    masstotal,mass0,xvec, avec=initcondition
    x0=xvec[1,0]
    y0=xvec[1,1]
    lastvalue=[1,1]
    thisvalue=[xvec[1,0],xvec[1,1]]
    i=1
    t=0
    while not (lastvalue[0]>0 and lastvalue[1]<0 and thisvalue[0]> 0 and thisvalue[1]>=0):
        reducedmass,xvec,avec,t=ODEeq.timestepRK4ODE(i,dt)
        if printoutput:
            print(xvec[1])
        #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
        #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
        #print(ax)
        xy=[xvec[1,0],xvec[1,1]]
        lastvalue,thisvalue=thisvalue, xy
        #lastvalue=thisvalue
        #thisvalue[0]=x[1]
        #thisvalue[1]=y[1]
        #thisvalue[0]=x[1]
        #thisvalue[1]=y[1]
        if printoutput:
            print(xy,lastvalue,thisvalue)
        i+=1
    
    print(thisvalue[0], thisvalue[1], lastvalue[0], lastvalue[1], x0, y0)
    return np.abs(x0-(thisvalue[0]+lastvalue[0])/2.)

In [15]:
dt=1 #*31556926 #seconds per year
numsteps=2000
mtotal,mass0,xvec,avec=xyuvaeqreducedmass
L0arr=[]
dtarr=[]
dtinit=0.125/4.
for i in np.arange(6):
    dt=(2**i)*dtinit
    dtarr.append(dt)
    L0=RunFirstOrbit(dt,xyuvaeqreducedmass,False)
    L0arr.append(L0)
print(dtarr, L0arr)

49.99999999953296 0.0002192859448121075 49.99999963631869 -0.0060307140405620845 50.0 0.0
49.99999988071706 0.003453927124506005 49.99999918169882 -0.009046072823391333 50.0 0.0
49.99999631800075 0.019188567850575995 49.99999966228481 -0.005811431665311891 50.0 0.0
49.99999901531124 0.009923209458593915 49.99998393852151 -0.04007678618495503 50.0 0.0
49.99997433783596 0.0506578421685326 49.99997565353719 -0.049342141156090036 50.0 0.0
49.99991647414743 0.09139244302314348 49.99988204414578 -0.10860742067949075 50.0 0.0
[0.03125, 0.0625, 0.125, 0.25, 0.5, 1.0] [1.820741744040788e-07, 4.6879205939376334e-07, 2.009857226425993e-06, 8.523083621980732e-06, 2.5004313428667047e-05, 0.00010074085339795147]


In [16]:
xdat=np.log10(np.array(dtarr))
ydat=np.log10(np.array(L0arr))

In [17]:
fitparam=np.polyfit(xdat,ydat,1)

In [18]:
fitparam

array([ 1.85300205, -4.01110848])

In [19]:
errfn=np.poly1d(fitparam)

In [20]:
errfn

poly1d([ 1.85300205, -4.01110848])

In [21]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="L0 error after one orbit, Circular, Vector, RK4", x_axis_label="dt", y_axis_label="L0", plot_width=400, plot_height=400)

# add a circle renderer with x and y coordinates, size, color, and alpha

p.circle(np.log10(np.array(dtarr)),np.log10(np.array(L0arr)), line_color="blue", legend="L0")
p.line(xdat,errfn(xdat),line_color="green", legend="Fit, dt^4.1")
p.legend.location = "bottom_right"
p.legend.click_policy="hide"

show(p) # show the

Loading BokehJS ...